In [1]:
import requests
import pandas as pd
import json
from base64 import b64encode
import datetime
from sqlalchemy import create_engine

# IMPORTANT!!! verify that it's pointed to the correct server
base_url = 'https://sierra-test.cincinnatilibrary.org:443/iii/sierra-api/v5/'
# base_url = 'https://sierra-test.cincinnatilibrary.org:443/iii/sierra-api/v5/'

# TODO: get the certs sorted out on the test server so we don't have to do this
# disable the warning message
# requests.packages.urllib3.disable_warnings() 

# client_key, client_secret, sierra_username, sierra_password
import patron_batch_upload_vars as vars

# create our connection string for connecting to our Sierra database
# NOTE: this isn't necessary (and unused in this example, but could be handy to also 
# use to check if barcode is already in use for example...
connection_uri = 'postgres://{}:{}@sierra-db.plch.org:1032/iii'.format(vars.sierra_username, vars.sierra_password)

# get access token and set the header for authentication
auth_string = b64encode(
    (vars.client_key + ':' + vars.client_secret).encode('utf-8')
).decode('utf-8')

# these values will be used for future requests (and may need to be refreshed)
access_headers = {}

def set_access_headers():
    """
    use this function to set and refresh the access_headers for future
    authorizing API requests 
    """
    headers = {}
    headers['authorization'] = 'basic ' + auth_string

    try:
        r = requests.post(base_url + 'token', headers=headers, verify=True)
    
    except requests.ConnectionError as e:
        print('connection error: {}'.format(e))
        return 0
    
    if r.status_code != 200:
        return 0

    access_token = r.json()['access_token']

    # set our headers to use the access token
    headers = {}
    headers['authorization'] = 'bearer ' + access_token
    headers['content-type'] = 'application/json'
    headers['accept'] = 'application/json'
    
    return headers


headers = set_access_headers()

if (headers):
    print('set_access_headers: success!')
else:
    print('set_access_headers: fail :(')

set_access_headers: success!


In [2]:
# here's a fake patron (Luigi) that was created via the Sierra SDA as a test.
# NOTE: we won't be able to set all these same fileds (`id`, 'updateDate', and
# 'createDate' among others can not be set) when we create our patron below,
# but it's still a good reference to have for the other fields.
url = base_url + 'patrons/' + str(2434753) + '?fields=,'
r = requests.get(url=url, headers=headers, verify=True)

# print(json.dumps(r.json(), indent=3,))

# just for display purpose, let's make it more colorful
from pygments import highlight, lexers, formatters
formatted_json = json.dumps(r.json(), indent=4)
colorful_json = highlight(formatted_json, lexers.JsonLexer(), formatters.TerminalFormatter())
print(colorful_json)

{
    "id": 2434753,
    "updatedDate": "2020-09-19T21:36:57Z",
    "createdDate": "2020-09-19T21:36:57Z",
    "deleted": false,
    "suppressed": false,
    "names": [
        "Mario, Luigi"
    ],
    "barcodes": [
        "01234567890"
    ],
    "expirationDate": "2023-09-19",
    "birthDate": "1985-09-13",
    "patronType": 0,
    "patronCodes": {
        "pcode1": "-",
        "pcode2": "-",
        "pcode3": 0,
        "pcode4": 0
    },
    "homeLibraryCode": "ba",
    "message": {
        "code": "-"
    },
    "blockInfo": {
        "code": "-"
    },
    "autoBlockInfo": {
        "code": "-"
    },
    "addresses": [
        {
            "lines": [
                "123 Mushroom Ln",
                "Cincinnati, OH 45236"
            ],
            "type": "a"
        }
    ],
    "phones": [
        {
            "number": "9999999999",
            "type": "t"
        }
    ],
    "moneyOwed": 0.0,
    "pMessage": "-",
    "fixedFields": {
        "43": {
            "labe

In [3]:
class PatronNew:
    """
    This class will create and defines a CUSTOMIZED patron record object for
    use with the Sierra REST API
    
    Creating the class sets values as defined arguments in the __init__
    function below
    
    The dictionary must be defined in a way that mirrors the patron object
    as defined by the sierra rest API. There are a few ways to determine
    how this should look to fit your needs:
    
        1. create a example patron in the sierra client, and then view how 
        REST API displays it. e.g.
        `https://example-library.edu:443/iii/sierra-api/v5/patrons/1042514?fields=,`

    """

    """
    pulling from a spreadsheet that has these fields
    Index(['First Name', 'Last Name', 'Barcode', 'Student ID', 'Alt ID',
       'School District', 'School', 'Birth Date', 'Email Address',
       'Home Phone Number ', 'Home Legal Address (no PO Box)',
       'Home Legal Address City', 'Home Legal Address Zip Code',
       'Home Mailing Address (if different than legal address)',
       'Home Mailing Address City', 'Home Mailing Address State',
       'Home Mailing Address Zip Code'],
      dtype='object')
    """
    def __init__(self,
                 last_name,
                 first_name,
                 birth_date,
                 phone_number,
                 email_address,
                 barcode,
                 addresses_line1,
                 addresses_line2,
                 home_library_code,
                 patron_agency,
                 school,
                 pin 
    ):
        
        # expiration date is 3 years from when these records are created
        # example:
        # expiration_date = '2023-09-03'
        expiration_date = (pd.Timestamp.now() + pd.DateOffset(years=3)).strftime('%Y-%m-%d')
        
        # here's an example, for our "school" cards, where we just set the
        # birthdate to be be the creation date minus 6 years
        #
        # birth_date = (pd.Timestamp.now() - pd.DateOffset(years=6))

        # here is another example where you can use the current date, and the
        # brithdate (if you supply it as an input) to determine what type of
        # patron you want to create. It uses uses the pd.Timedelta object to
        # determine the number of years old at the time of record creation.
        #
        # self.years_old = int( (pd.Timestamp.now() - birth_date).days / 365)
        # if (self.years_old >= 18):
        #     self.patron_type = 3
        # elif (self.years_old >= 13):
        #     self.patron_type = 2
        # else:
        #     self.patron_type = 1

        # all child-only here
        self.patron_type = 1
            
        # normalize the school name. In case school comes in as something like:
        # "Tyler elementry school" will be converted to:
        # "Tyler Elementry School"
        self.school = school.lower().title()

        # address data:
        # here's an example if each part was broken up:
        # addresses_line1 = address
        # addresses_line2 = city + ', ' + state + ' ' + zip_code
        #
        # otherwise, we just bring in the address lines as we would expect
        
        # this is the data object that matches the patronPatch object
        self.patron_data = {
            
            # EXAMPLE of how to formatat dates coming in as '%m-%d-%Y', and converted to %Y-%m-%d (isoformat)
            #"expirationDate": str(datetime.strptime(str(expiration_date), "%m-%d-%Y").date().isoformat()),
            'expirationDate': expiration_date,
            'birthDate': birth_date.strftime('%Y-%m-%d'),
            'patronType': self.patron_type,
            'blockInfo': {'code': '-'},
            'phones': [{'number': str(phone_number), 'type': 't'}],
            'emails': [str(email_address)],
            'pMessage': '-',
            'fixedFields': {
                '44': {'label': 'E-Lib Update? (P1)', 'value': 'n'},
                '45': {'label': 'Friends? (P2)', 'value': 'n'},
                '46': {'label': 'Foundation? (P3)', 'value': '1'},
                '86': {'label': 'Agency', 'value': '1', 'display': 'Main Library'},
                '158': {'label': 'Patron Agency', 'value': '1'},
                '268': {'label': 'Notice Preference', 'value': 'z'}
            },
            
            "names": [
                str(last_name) + ", " + str(first_name)
            ],
            "barcodes": [
                str(barcode)
            ],
            "homeLibraryCode": str(home_library_code),
            "varFields": [
                # field tag d = birthDate (MMDDYYYY)
                {
                    'fieldTag': 'd',
                    'content': birth_date.strftime('%m%d%Y')
                    # 'content': '09152015'
                },
                # set a note field
                {
                    "fieldTag": 'x',
                    "content": 'ConnectED'
                },
                {
                    "fieldTag": "l",
                    "content": self.school
                },
            ],
            "addresses": [
                {
                    "lines": [
                        str(addresses_line1),
                        str(addresses_line2)
                    ],
                    "type": "a"
                },
                # possibly append to addresses below...
            ],

            "pin": str(pin),
        }
        
        
        # at this point, our patron record is complete. If you wanted to
        # programatically add or delete from the record based on some
        # conditions, you can do so here ... for example, if we wanted
        # to set a second address for data we perhaps took in as an additional
        # variable:
        #
        # test if address2 is not the same as address1 ... 
        # if they're different, append it to the addresses
        #
        #if ( (addresses_2_line1 + addresses_2_line2) != (addresses_line1 + addresses_line2) ):
        #    self.patron_data['addresses'].append(
        #        {
        #            "lines": [
        #                str(addresses_2_line1),
        #                str(addresses_2_line2)
        #            ],
        #            "type": "h"
        #        },
        #    
        #    )
                


    def get_dict(self):
        '''
        this will simply return the patron data set up in the local dictionary
        '''
        return self.patron_data

In [4]:
# read from the Excel file into a Pandas Dataframe
# NOTE: convert Card# to string, so that it preserves a leading zero.
# You may need to add additional converters here as necessary to force the
# dataframe to treat fields as certain datatypes.  
patron_input = pd.read_excel('2020-09-19-example_data.xlsx', converters={'Card#':str})

# examine the df
print(patron_input)

  Last Name First Name Birth Date      Phone#               E-mail  \
0     Mario      Mario 1985-09-13  9999999999  mario@nintendo.com    
1     Peach   Princess 1985-09-14  9999999999  peach@nintendo.com    
2      Toad    Captain 1985-09-15  9999999999   toad@nintendo.com    

         Card#          Address       City, State Zip Home Library  \
0  01234567891  123 Mushroom Ln  Cincinnati, OH 45236           ba   
1  01234567892      1 Castle Ln  Cincinnati, OH 45236           ba   
2  01234567893  125 Mushroom Ln  Cincinnati, OH 45236           ba   

   Patron Agency               School  
0              1  Mushroom highschool  
1              1  Mushroom Highschool  
2              1  Mushroom Highschool  


In [5]:
patron_input.keys()

Index(['Last Name', 'First Name', 'Birth Date', 'Phone#', 'E-mail', 'Card#',
       'Address', 'City, State Zip', 'Home Library', 'Patron Agency',
       'School'],
      dtype='object')

In [6]:
# NOTE: the birthdate has been pulled in as a datetime64 datatype, which is
# exaclty what we wanted
patron_input.dtypes

Last Name                  object
First Name                 object
Birth Date         datetime64[ns]
Phone#                      int64
E-mail                     object
Card#                      object
Address                    object
City, State Zip            object
Home Library               object
Patron Agency               int64
School                     object
dtype: object

In [7]:
# capture the output so we can do some other stuff with it
output = ''

for i, row in patron_input.iterrows():
    patron_obj = PatronNew(
        last_name = row['Last Name'],
        first_name = row['First Name'],
        birth_date = row['Birth Date'],
        phone_number = row['Phone#'],
        email_address = row['E-mail'],
        barcode = row['Card#'],
        addresses_line1 = row['Address'],
        addresses_line2 = row['City, State Zip'],
        home_library_code = row['Home Library'],
        patron_agency = row['Patron Agency'],
        school = row['School'],
        # just hardcode the pin, but this could easily come from the
        # spreadsheet.
        pin = '1234'
    )
    
    # here's where the data is actually created in Sierra. You may want to take
    # special note of the patron record numbers so that you may take the data
    # and run it through create lists later on:
    # NOTE, for especially large lists, you may want to either keep a timer, or
    # just refresh the access token every few hundres records so that you don't
    # end up with an expired access token
    timestamp = datetime.datetime.now(tz=datetime.timezone.utc).isoformat()
    r = requests.post(base_url + 'patrons/', headers=headers, data=( json.dumps(patron_obj.get_dict()) ), verify=True)
    
    row_output = '{}\t{}\t{}\t{}'.format(
        timestamp,
        i,
        r.status_code,
        r.text
    )

    output += row_output + '\n'
    
    print(row_output, flush=True, end='\n')

2020-09-19T22:48:48.552022+00:00	0	200	{"link":"https://sierra-test.cincinnatilibrary.org/iii/sierra-api/v5/patrons/2434760"}
2020-09-19T22:48:49.472647+00:00	1	200	{"link":"https://sierra-test.cincinnatilibrary.org/iii/sierra-api/v5/patrons/2434761"}
2020-09-19T22:48:50.382563+00:00	2	200	{"link":"https://sierra-test.cincinnatilibrary.org/iii/sierra-api/v5/patrons/2434762"}


In [8]:
# this is kinda a slap-dash way to extract the patron record from the url
import re
search_regex = re.compile(
    r"(?:^.*/)([0-9]{7})(?:\"})"
)

for line in output.split('\n'):
    #print(line)
    x = None
    try:
        x = search_regex.search(line).groups()[0]
    except:
        pass
    if x:
        print('p{}a'.format(x), end=',\n')


p2434760a,
p2434761a,
p2434762a,
